# Persona & Role‑Play Workbench  
Create multi‑persona scenarios, set individual tones, reading levels, and length limits, then run a simulated dialogue.

In [ ]:
!pip -q install openai ipywidgets textstat

In [ ]:
import os, json, openai, ipywidgets as w, textstat
from IPython.display import display, Markdown, clear_output

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'sk-')

# --- Widgets ---
scenario_box = w.Textarea(
    value="Topic: Explaining healthy sleep habits",
    description="Scenario:",
    layout=w.Layout(width='100%', height='60px'))

n_roles = w.IntSlider(value=2, min=1, max=5, step=1, description='Roles')

# dynamic boxes for persona definitions
role_boxes = []
def rebuild_roles(change=None):
    global role_boxes
    for rb in role_boxes:
        rb.close()
    role_boxes = []
    for i in range(n_roles.value):
        rb = w.Textarea(
            value=f"Role {i+1}:\nName: Person{i+1}\nTone: formal\nReading_Level: 10\nLength: <=50 words",
            layout=w.Layout(width='100%', height='80px'))
        role_boxes.append(rb)
    roles_vbox.children = role_boxes

roles_vbox = w.VBox()
n_roles.observe(rebuild_roles, names='value')
rebuild_roles()

run_btn = w.Button(description='Simulate 👉')
out = w.Output()

def parse_roles():
    roles = []
    for rb in role_boxes:
        lines = [l.strip() for l in rb.value.splitlines() if l.strip()]
        role_dict = {}
        for ln in lines:
            if ':' in ln:
                k, v = ln.split(':', 1)
                role_dict[k.strip().lower()] = v.strip()
        roles.append(role_dict)
    return roles

def build_prompt(roles, scenario):
    sys_msg = "You are participating in a role‑play conversation. Follow persona definitions strictly."
    msgs = [{'role':'system','content': sys_msg},
            {'role':'system','content': f"Scenario: {scenario}"}]
    # prime each persona
    for r in roles:
        msgs.append({'role':'system','content':
                     f"Persona {r.get('name','')} — Tone: {r.get('tone','')}; "
                     f"Reading_Level: {r.get('reading_level','')}; Length limit: {r.get('length','')}"})
    # kick‑off
    msgs.append({'role':'user','content': "Begin the conversation."})
    return msgs

def run_sim(b):
    with out:
        out.clear_output()
        roles = parse_roles()
        msgs = build_prompt(roles, scenario_box.value)
        print("Prompt:", json.dumps(msgs, indent=2))
        try:
            resp = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=msgs,
                temperature=0.7,
                max_tokens=400
            )
            text = resp.choices[0].message.content
            display(Markdown("### 🗣️ Conversation\n" + text))
            # readability
            grade = textstat.flesch_kincaid_grade(text)
            print(f"Flesch‑Kincaid Grade Level ≈ {grade:.1f}")
        except Exception as e:
            print("Error:", e)

run_btn.on_click(run_sim)

ui = w.VBox([scenario_box, n_roles, roles_vbox, run_btn, out])
display(ui)
print("Define personas, then **Simulate**!")


---  
### Tips  
* Use explicit delimiters like `<<Patient>>` / `<<Doctor>>` inside personas.  
* Combine with length limits for concise turn‑taking.  
* Evaluate reading level to match your audience.